In [ ]:
# coding: utf-8
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model
from keras.models import model_from_json
from keras.optimizers import RMSprop
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

In [ ]:
from pelops.datasets.featuredataset import FeatureDataset
from pelops.datasets.veri import VeriDataset
from pelops.experiment_api.experiment import ExperimentGenerator
from pelops.analysis import analysis
from pelops.analysis.camerautil import get_match_id, make_good_bad
import pelops.utils as utils
import numpy as np

import os
import glob
import random
import multiprocessing as mp
import queue
import threading
import numpy as np
import datetime

In [ ]:
# get a GPU session and reserve memory
def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# load an image from disk
def load_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


# use an image with a model to get features
def image_features(img, model, length=2048):
    features = np.zeros((1, length), dtype=np.float16)
    #model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    predictions = model.predict(img)
    return predictions

In [ ]:
def save_model_workaround(model, model_output_file, weights_output_file):
    print('saving model   to {}'.format(model_output_file))
    print('saving weignts to {}'.format(weights_output_file))
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_output_file, 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weights_output_file)


def load_model_workaround(model_output_file, weights_output_file):
    # load json and create model
    json_file = open(model_output_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_output_file)
    return loaded_model

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    #dont be stupid and use the wrong axis..
    # return K.sqrt(K.sum(K.square(x - y), axis=3, keepdims=True))
    return K.sqrt(K.sum(K.square(x - y), axis=3, keepdims=False))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels, threshold=0.5):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < threshold].mean()

In [ ]:
def make_examples(gen,examples):
    left = list()
    right = list()
    truth = list()
    
    for _ in range(examples):
        cameras = gen.generate()
        match_id = get_match_id(cameras)
        goods, bads = make_good_bad(cameras,match_id)
        left.append(goods[0])
        right.append(goods[1])
        left.append(bads[0])
        right.append(bads[1])
        truth.append(True)
        truth.append(False)
    return(left,right,truth)

def examples_2_images(left,right):
    length = len(left)
    left_images = np.zeros((length,299,299,3))
    right_images = np.zeros((length,299,299,3))
    for idx,work in enumerate(left):
        filename = work.filepath
        left_images[idx] = load_image(filename)
    for idx,work in enumerate(right):
        filename = work.filepath
        right_images[idx] = load_image(filename)
    return left_images, right_images

In [ ]:
#do the keras setup

In [ ]:
KTF.set_session(get_session(.3))

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
input_left = Input(shape=(299, 299, 3))
input_right = Input(shape=(299, 299, 3))

In [ ]:
processed_left = base_model(input_left)
processed_right = base_model(input_right)

In [ ]:
distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_left, 
                                                        processed_right])

In [ ]:
model = Model([input_left, input_right], distance)

In [ ]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)

In [ ]:
#get ready to make data and try the model

In [ ]:
#set some constants
ITEMSPERCAMERA = 2
YRANDOM=1024
CAMERAS=2
DROPPED=0
CMC=100
EXPERIMENTS=4000

In [ ]:
veri_validate = VeriDataset('/local_data/dgrossman/VeRi',set_type=utils.SetType.TEST.value)
veri_train = VeriDataset('/local_data/dgrossman/VeRi',set_type = utils.SetType.TRAIN.value)

In [ ]:
expGen_validate = ExperimentGenerator(veri_validate, CAMERAS, ITEMSPERCAMERA, DROPPED, YRANDOM)
expGen_train = ExperimentGenerator(veri_train, CAMERAS, ITEMSPERCAMERA, DROPPED, YRANDOM)

In [ ]:
left_train,right_train,truth_train = make_examples(expGen_train,EXPERIMENTS)

In [ ]:
left_validate,right_validate,truth_validate = make_examples(expGen_validate,EXPERIMENTS)

In [ ]:
left_images_train, right_images_train = examples_2_images(left_train,right_train)
left_images_validate, right_images_validate = examples_2_images(left_validate,right_validate)

In [ ]:
epochs = 20
full_history = model.fit([left_images_train, right_images_train], np.array(truth_train),
                         validation_data=([left_images_validate, right_images_validate], np.array(truth_validate)),
                         batch_size=16,
                         epochs=epochs)